In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


df_titanic = sns.load_dataset('titanic')
df_titanic.rename(columns={'fare': 'target'}, inplace=True)
df_titanic_num = df_titanic.select_dtypes(include=[np.number])

# Imputation and Basic Pipelines

## Imputation

Until this point, if we are missing data in our dataset, we have simply dropped the rows with missing data. However, this is not always the best approach. In real world data analysis, we often need to deal with missing data without just losing everything. For example, if your company does a survey of customers to collect data, it is likely that at least some of the customers will not answer all the questions. Just dropping an entire row of data if one question is missing can be costly - getting this data takes effort and costs money, so we want to use it as much as possible.

Imputation is the process of replacing missing data with substituted values, or deleting it. There are several approaches, from dead simple to complex and advanced. 

### Simple Imputation

The tool that we can use to do imputation is the sklearn `SimpleImputer`. This works fairly simply, it takes in a dataset and replaces all missing values with a specified value. 

### Removal

The most simple version of imputation is to just remove the rows with missing data. 

### Numerical Imputation

When we have numerical data, we can replace missing values with the mean, median, or mode of the column.

#### Mean vs Median

For most numerical values, we use either the mean or the median if we are inserting a value, at least for simple imputation. The distribution of a column can offer some guidance on which to use. 

One common example of this is with income/wealth related data, where we have a distribution that tends to be very skewed. If using skewed data, the median can often be a better choice than the mean for imputation, as the mean can be substantially impacted by some very large or very small values. This is very situation dependent, and you should always consider the specifics of your data.

### Category Imputation

When we have categorical data, we can replace missing values with the most common category, or a new category that represents missing data.

### Imputation Considerations


#### Removing vs Imputing

One common consideration if a feature has many missing values is whether to remove the feature or impute the missing values. If a feature has many missing values, it may not be useful for analysis. However, if the feature is important, it may be better to impute the missing values. There is a trade off here that will depend on the specific dataset and analysis. As a rule of thumb, if a feature has more than 15% missing values, it may be better to remove the feature.

## Smarter Imputation

There are more advanced methods for imputation, such as using another machine learning models to predict the missing values. The idea here doesn't change, we have values that are missing, and we want to replace them with something that allows us to keep the value and use it in our analysis. The main thing that differs is that the simple imputation uses some pretty basic logic to replace the missing values, while the more advanced methods use more complex analysis to try to generate a better replacement for the missing value.

## Pipelines

Pipelines are a very useful tool because they make our lives much easier when doing some of these data preparation steps. A pipeline is a series of steps that are executed in order. This is useful because it allows us to do a series of steps in a single command, and it also allows us to do the same steps on multiple datasets. We will create a pipeline that contains multiple steps, then instead of using a model's .fit() method to train the model on our processed data, we'll use the .fit() method on the pipeline to train the model on our data after all those steps we define are done. We basically can add many of the preparation steps to the pipeline, then use it just as we'd use a model, and the pipeline will automatically scale, impute, or do whatever else we've defined in the pipeline.

### Sklearn Pipelines

The pipeline tool that we use in sklearn to do this is the `Pipeline` tool. This tool allows us to define a series of steps, then use the pipeline as if it were a model. The creation and setup is fairly simple, we list the steps in a list of tuples, where the first element of the tuple is the name of the step, and the second element is the step itself. We then pass this list to the `Pipeline` tool, and we can use the pipeline as if it were a model.

In [7]:
df_titanic_num.head()

,survived,pclass,age,sibsp,parch,target
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [8]:
pipe = Pipeline([

])

### Column Transformer Introduction

One limitation of the pipeline tool is that it just does what it does to whatever you feed it. This is fine if you have a dataset that is all numerical, but if you have a dataset where we need to do encoding on some values and do scaling on some others, this isn't so simple. The `ColumnTransformer` tool is a tool that allows us to do different things to different columns. We can use this to do encoding on some columns and scaling on others, then pass this to the pipeline tool to do all of these steps in a single command.

The column transformer can be a little annoying to construct syntax-wise, but the idea is very simple - it is a pipeline that can define different routes for different columns in our data. 

#### Sklearn Column Transformer

Using the `ColumnTransformer` tool is very similar to using the `Pipeline` tool. We define a list of tuples, where the first element of the tuple is the name of the step, and the second element is the step itself. We then pass this list to the `ColumnTransformer` tool, and we can use the column transformer as if it were a model.

In [9]:
df_titanic.head()


,survived,pclass,sex,age,sibsp,parch,target,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


##### Splitting Preparation

In this dataset we have a mix of numerical and categorical data. We will use the `ColumnTransformer` to do different things to different columns. We can define "lanes" of preparation that needs to take place, and then place each column into the correct lane. We need to:
<ul>
<li> Numerical data - impute missing values with the median, then scale the data. </li>
<li> Categorical data - impute missing values with the most common category, then encode the data. </li>
</ul>

Each of these can be their own pipeline, they will do their process just like a normal pipeline, they'll just be applied to a subset of the original columns. The column transformer definition will have 3 parts - the name of the step, the step itself, and the columns that the step will be applied to.

In [10]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

ct = ColumnTransformer([
    ('num', num_pipe, ['age', 'pclass', 'sibsp', 'parch', 'target']),
    ('cat', cat_pipe, ['sex', 'class', 'embarked', 'who', 'deck', 'embark_town', 'alive', 'alone'])
])

#### Using the Column Transformer

Once the column transformer and its pipelines are created, they again work just like a normal model. 

## Using Pipelines in Practice

In real applications, these pipelines make our lives far easier. Pipelines and column transformers can be stacked, mixed, and combined to make a set of steps that can do almost anything we desire. Most commonly, we may have different steps for different columns, and we have a few parallel pipelines that do different things to different columns. For example, we may do imputation differently on different sets of numerical columns and do encoding differently on different sets of categorical columns. Then we may combine all 4+ sets of steps using the column transformer. 